In [ ]:
# Imports
from fucciphase import process_dataframe
from fucciphase.phase import NewColumns, compute_cell_cycle, generate_cycle_phases
from fucciphase.plot import plot_phase
from fucciphase.utils import normalize_channels, simulate_single_track

## Generate data

The next cell generates a dummy dataset for the purpose of showing how fucciphase
works.

In [ ]:
df = simulate_single_track()
print(df)

## Normalize the last two channels

Next, we normalize the channels. We need to pass the channel names to the function.
The normalization function also runs a moving average by default on each track.

In [ ]:
new_channels_name = normalize_channels(df, ["MEAN_INTENSITY_CH3", "MEAN_INTENSITY_CH4"])
print(df[new_channels_name])

## Compute the cell cycle percentage for each spot

For each spot, we compute the cell cycle percentage using a trigonometric approach.

In [ ]:
new_channels_name = compute_cell_cycle(df, "MEAN_INTENSITY_CH3", "MEAN_INTENSITY_CH4")
print(df[new_channels_name])

## Identify the cell cycle phases

If you know where the thresholds for the different phases are, you can also use the 
cell cycle percentage to identify the phase. You need to provide one more phase than
thresholds, the first threshold corresponding to the end of the first phase. 

In [ ]:
generate_cycle_phases(df, phases=["G1", "T", "S", "M"], thresholds=[0.04, 0.44, 0.56])
print(df[[NewColumns.cell_cycle(), NewColumns.phase()]])

## Run all at once

The whole process can also be run at once using the `process_dataframe` function.

There are additional parameters to tune the analysis. Check out the function 
documentation.

In [ ]:
df = simulate_single_track()
process_dataframe(
    df,
    "MEAN_INTENSITY_CH3",
    "MEAN_INTENSITY_CH4",
    phases=["G1", "T", "S", "M"],
    thresholds=[0.04, 0.44, 0.56],
)

## Plot results

We provide a convenience function to plot the results.

In [ ]:
plot_phase(df, "MEAN_INTENSITY_CH3", "MEAN_INTENSITY_CH4")